<a href="https://colab.research.google.com/github/yavuzuzun/projects/blob/main/validity_of_sampling_on_the_detection_of_a_dynamical_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following code uses a generative model(Izhikevich model) for neural(spiking) activity produced by a dynamical system (phasic spiking) for arbitrary population size and makes sampling over that population for changing noise level and sampling size. Later, I generated null model - time series for these samplings enforcing the same firing rate and leaving the spiking time of neurons arbitrary and independent from each other. In the following step, I trained a random forest regressor to predict if a sample time series coming from a dynamical system or not. Success rates of the model changes with the changing sampling size and noise level. This model can be improved by including various dynamical model and different sampling methods. In this version it offers a template for a more in depth analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

# set the seed to a fixed value
random.seed(333)

# Define the simulation parameters
dt = 0.1 # time step (ms)
T = 1000 # simulation time (ms)
N = 100 # number of neurons in the population
M = 100 # number of time series 

# Define the Izhikevich neuron model parameters
a_vals = 0.02 + np.random.normal(0,1,M) * 0.002
b_vals = 0.25 + np.random.normal(0,1,M) * 0.0025
c_vals = -65 + np.random.normal(0,1,M) * 6.5
d_vals = 6 + np.random.normal(0,1,M) * 0.6


# Define the noise parameters
mu = 0 # mean of the noise
sigma = 4 # standard deviation of the noise

activity_list1 = []

for i in range(M):

  # print(i) # keep track of the progress

  # Assign current model parameters
  a = a_vals[i]
  b = b_vals[i]
  c = c_vals[i]
  d = d_vals[i]

  # Initialize the neuron parameters
  v = np.random.rand(N) * 30 - 70 # initial membrane potential (mV)
  u = b * v # initial recovery variable

  # Define the input to the population
  I = np.zeros((N, int(T/dt)))
  I[:, 0:int(T/dt)] = 1 # external input for first half of simulation

  # Simulate the neuron population with added noise
  for j in range(1, int(T/dt)):
      # Compute the input current to each neuron
     input_current = I[:, j] + np.dot(np.random.rand(N), 5) # random background input
     noise = np.random.normal(mu, sigma, N) # generate noise for each neuron
    
      # Update the membrane potential and recovery variable with added noise
     v += dt * (0.04 * v**2 + 5 * v + 140 - u + input_current + noise)
     u += dt * a * (b * v - u)
    
      # Apply spike threshold and reset
     spikes = np.where(v >= 30)[0]
     v[spikes] = c
     u[spikes] += d
    
      # Store the neuron activity
     if j == 1:
         activity = np.zeros((N, int(T/dt)))
     activity[:, j] = np.in1d(range(N), spikes).astype(int)

  activity_list1.append(activity)

In [ ]:
mean_firingRate = np.mean(activity_list1)
shape_timeSeries = activity_list1[0].shape

for i in range(M):
  # generate a new time series with the same shape as 'original_ts'
  arr = np.random.rand(shape_timeSeries[0],shape_timeSeries[1])
  arr = (arr < mean_firingRate).astype(int)
  activity_list1.append(arr)


In [ ]:
size_subsampling = 10
repetation = 10

subsample_1_10 = []
for i in range(np.shape(activity_list1)[0]):
  for j in range(repetation):
    indices = random.sample(range(N), size_subsampling)
    subsample_1_10.append(activity_list1[i][indices])


In [ ]:
np.shape(subsample_1_10)

(2000, 10, 10000)

In [ ]:
def get_corrValues(time_series):
  pop_size = np.shape(time_series)[0]
  corr_m = np.corrcoef(time_series)
  return corr_m, corr_m[np.triu_indices(pop_size, k=1)].tolist(), 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import networkx as nx

# Define Gaussian filter with sigma=3
sigma = 200 # gaussian filter parameter, 20 ms
window_size = sigma * 6 + 1
window = signal.gaussian(window_size, sigma)

length_corrVals = 61
cc_threshold = 0.2

corr_array = np.zeros([np.shape(subsample_1_10)[0], length_corrVals])
for i in range(np.shape(subsample_1_10)[0]):
  timeS = subsample_1_10[i]
  for j in range(np.shape(timeS)[0]):
    timeS[j] = np.convolve(timeS[j], window, mode='same')
  corr_matrix, corr_list = get_corrValues(timeS)
  corr_matrix[corr_matrix < cc_threshold] = 0

  # get correlation values from the upper triangle of the correlation matrix
  corr_array[i][15:-1] = np.sort(corr_list)
  corr_array[i][-1] = np.mean(corr_list)
  
  eigenvalues = np.linalg.eigvals(corr_matrix)
  # sort the eigenvalues in descending order
  sorted_eigenvalues = np.sort(eigenvalues)[::-1]
  # take the first three eigenvalues
  corr_array[i][0:3] = sorted_eigenvalues[:3]
  
  G = nx.from_numpy_array(corr_matrix)
  # calculate the degree distribution
  degree_hist = nx.degree_histogram(G)
  corr_array[i][3:3+len(degree_hist)] = degree_hist

np.var(corr_array,axis=1)

<ipython-input-14-b661ceb5e58f>:30: ComplexWarning: Casting complex values to real discards the imaginary part
  corr_array[i][0:3] = sorted_eigenvalues[:3]


array([0.59711051, 0.49101741, 0.64137867, ..., 0.92978159, 0.54878712,
       0.56241084])

In [ ]:
np.shape(corr_array)

(2000, 61)

In [ ]:
import pandas as pd

dyn_df = pd.DataFrame(corr_array)
dyn_df

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,3.170875,1.873277,1.217114,0.0,0.0,1.0,0.0,0.0,2.0,2.0,...,0.446060,0.465313,0.467453,0.496267,0.532848,0.557590,0.590216,0.599995,0.611870,0.156248
1,2.370547,1.704031,1.454965,0.0,0.0,0.0,1.0,2.0,4.0,1.0,...,0.321299,0.322379,0.336295,0.348735,0.400081,0.432907,0.463864,0.528448,0.626083,0.123604
2,3.573562,1.851036,1.019046,0.0,0.0,1.0,0.0,2.0,0.0,1.0,...,0.498026,0.501470,0.501683,0.523120,0.532077,0.591626,0.628506,0.672810,0.797521,0.168611
3,3.461411,1.302575,1.106696,0.0,0.0,2.0,1.0,1.0,0.0,2.0,...,0.498119,0.517514,0.519248,0.563302,0.563624,0.641508,0.644568,0.661129,0.811936,0.146179
4,2.675328,2.041719,1.236895,0.0,0.0,0.0,2.0,1.0,1.0,3.0,...,0.357431,0.423227,0.425761,0.439405,0.456264,0.571026,0.599995,0.628506,0.645333,0.149580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.657622,1.594613,1.500934,0.0,0.0,0.0,6.0,4.0,0.0,0.0,...,0.168145,0.170317,0.285075,0.286473,0.317719,0.387285,0.409251,0.491411,0.594613,0.024673
1996,1.956757,1.482395,1.278332,0.0,0.0,1.0,4.0,1.0,4.0,0.0,...,0.225275,0.245129,0.252154,0.320900,0.346750,0.357687,0.407650,0.435454,0.451962,0.015380
1997,1.689221,1.549840,1.508417,0.0,0.0,0.0,4.0,6.0,0.0,0.0,...,0.179963,0.266035,0.271198,0.295870,0.348950,0.361843,0.410590,0.432042,0.509045,0.039890
1998,1.841913,1.459128,1.048905,0.0,0.0,4.0,2.0,2.0,2.0,0.0,...,0.174416,0.194696,0.198630,0.201448,0.346162,0.370266,0.375231,0.455239,0.461234,-0.007342


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Split the dataset into training and testing sets
number_subsample = M*repetation
X = dyn_df.iloc[:,15:]  # Features (remove the target variable column)
y = np.concatenate((np.ones([number_subsample,1]), np.zeros([number_subsample,1])))              # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Create a Random Forest Regressor with default parameters
rf = RandomForestRegressor()

# Train the model on the training set
rf.fit(X_train, y_train)

# Use the trained model to make predictions on the testing set
y_pred = rf.predict(X_test)

# Evaluate the model's performance using mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print('Mean squared error:', mse)

<ipython-input-17-efc043fa9332>:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


Mean squared error: 0.050056800000000005


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Split the dataset into training and testing sets
number_subsample = M*repetation
X = dyn_df.iloc[:,:15]  # Features (remove the target variable column)
y = np.concatenate((np.ones([number_subsample,1]), np.zeros([number_subsample,1])))              # Target variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Create a Random Forest Regressor with default parameters
rf = RandomForestRegressor()

# Train the model on the training set
rf.fit(X_train, y_train)

# Use the trained model to make predictions on the testing set
y_pred = rf.predict(X_test)

# Evaluate the model's performance using mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print('Mean squared error:', mse)

<ipython-input-18-cb8426f1dc23>:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


Mean squared error: 0.05820619999999999
